In [4]:
!pip install qiskit_aer

  Obtaining dependency information for qiskit_aer from https://files.pythonhosted.org/packages/a3/4a/f40f4655010b104e4e98a89f13e960fb6f02f2b2ceb6ecf73762bce86d22/qiskit_aer-0.15.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB 320.0 kB/s eta 0:00:30
   ---------------------------------------- 0.0/9.5 MB 326.8 kB/s eta 0:00:29
   ---------------------------------------- 0.1/9.5 MB 491.5 kB/s eta 0:00:20
    --------------------------------------- 0.1/9.5 MB 653.6 kB/s eta 0:00:15
   - -------------------------------------- 0.3/9.5 MB 1.1 MB/s eta 0:00:09
   - -------------------------------------- 0.4/9.5 MB 1.2 MB/s eta 0:00:08
   -- ------------------------------------- 0.5/9.5 MB 1.5 MB/s eta 0:00:07
   -- ------------------------------------- 0.7/9.5 MB 1.6 MB/s eta 0:00:06
   --- ------------------------

In [27]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.result import marginal_distribution
from qiskit.circuit.library import UGate
import numpy as np
import random
import math

In [28]:
qubit = QuantumRegister(1,"Q")
ebit0 = QuantumRegister(1,"A")
ebit1 = QuantumRegister(1,"B")
a = ClassicalRegister(1, "a")
b = ClassicalRegister(1, "b")
#let the name of our circuit be 'protocol'
protocol = QuantumCircuit(qubit, ebit0, ebit1, a, b)
#preparing ebit for teleportation
protocol.h(ebit0)
protocol.cx(ebit0, ebit1)
protocol.barrier()

#alice apply the operations
protocol.cx(qubit, ebit0)
protocol.h(qubit)
protocol.barrier()

#alice measure and send clasical bits to bob
protocol.measure(qubit,b)
protocol.measure(ebit0,a)
protocol.barrier()

#bob apply the operation after checking
#since the if_test function returns an handle we use with block
with protocol.if_test((a,1)):
    protocol.x(ebit1)
with protocol.if_test((b,1)):
    protocol.z(ebit1)
display(protocol.draw())


░      ┌───┐ ░ ┌─┐    ░                                     »
  Q: ───────────░───■──┤ H ├─░─┤M├────░─────────────────────────────────────»
     ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░                                     »
  A: ┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░─────────────────────────────────────»
     └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░ ┌────── ┌───┐ ───────┐ ┌────── ┌───┐»
  B: ─────┤ X ├─░────────────░──╫──╫──░─┤ If-0  ┤ X ├  End-0 ├─┤ If-0  ┤ Z ├»
          └───┘ ░            ░  ║  ║  ░ └──╥─── └───┘ ───────┘ └──╥─── └───┘»
                                ║  ║    ┌──╨──┐                   ║         »
a: 1/═══════════════════════════╬══╩════╡ 0x1 ╞═══════════════════╬═════════»
                                ║  0    └─────┘                ┌──╨──┐      »
b: 1/═══════════════════════════╩══════════════════════════════╡ 0x1 ╞══════»
                                0                              └─────┘      »
«               
«  Q: ──────────
«               
«  A: ──────────
«      ───────┐ 
«  B:   End-0 ├─
«      ───────┘ 
«a: 1/══════════
«               
«b: 1/══════════
«

In [32]:
pi = math.pi
random_gate = UGate(
    theta = random.random() * 2 * pi,
    phi = random.random()* 2* pi,
    lam = random.random() * 2* pi
)
display(random_gate.to_matrix())

array([[-0.49085359+0.j        ,  0.41235895-0.76747824j],
       [-0.39235733-0.77789362j, -0.49068916-0.01270403j]])

In [18]:
help(QuantumCircuit.if_test)

Help on function if_test in module qiskit.circuit.quantumcircuit:

if_test(self, condition, true_body=None, qubits=None, clbits=None, *, label=None)
    Create an ``if`` statement on this circuit.
    
    There are two forms for calling this function.  If called with all its arguments (with the
    possible exception of ``label``), it will create a
    :obj:`~qiskit.circuit.IfElseOp` with the given ``true_body``, and there will be
    no branch for the ``false`` condition (see also the :meth:`.if_else` method).  However, if
    ``true_body`` (and ``qubits`` and ``clbits``) are *not* passed, then this acts as a context
    manager, which can be used to build ``if`` statements.  The return value of the ``with``
    statement is a chainable context manager, which can be used to create subsequent ``else``
    blocks.  In this form, you do not need to keep track of the qubits or clbits you are using,
    because the scope will handle it for you.
    
    For example::
    
        from qis

In [39]:
#applying the random gate to o qubit Q
#creating a new circuit
test = QuantumCircuit(qubit, ebit0, ebit1, a,b)
# Start with the randomly selected gate on Q

test.append(random_gate, qubit)
test.barrier()

# Append the entire teleportation protocol from above.

test = test.compose(protocol)
test.barrier()

# Finally, apply the inverse of the random unitary to B and measure.

test.append(random_gate.inverse(), ebit1)

result = ClassicalRegister(1, "Result")
test.add_register(result)
test.measure(ebit1, result)

display(test.draw())

┌─────────────────────────┐ ░            ░      ┌───┐ ░ ┌─┐    ░ »
       Q: ┤ U(4.1677,4.2452,2.0638) ├─░────────────░───■──┤ H ├─░─┤M├────░─»
          └─────────────────────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░ »
       A: ────────────────────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░─»
                                      ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░ »
       B: ────────────────────────────░──────┤ X ├─░────────────░──╫──╫──░─»
                                      ░      └───┘ ░            ░  ║  ║  ░ »
     a: 1/═════════════════════════════════════════════════════════╬══╩════»
                                                                   ║  0    »
     b: 1/═════════════════════════════════════════════════════════╩═══════»
                                                                   0       »
Result: 1/═════════════════════════════════════════════════════════════════»
                                                                           »
«                                                         ░ »
«       Q: ───────────────────────────────────────────────░─»
«                                                         ░ »
«       A: ───────────────────────────────────────────────░─»
«          ┌────── ┌───┐ ───────┐ ┌────── ┌───┐ ───────┐  ░ »
«       B: ┤ If-0  ┤ X ├  End-0 ├─┤ If-0  ┤ Z ├  End-0 ├──░─»
«          └──╥─── └───┘ ───────┘ └──╥─── └───┘ ───────┘  ░ »
«          ┌──╨──┐                   ║                      »
«     a: 1/╡ 0x1 ╞═══════════════════╬══════════════════════»
«          └─────┘                ┌──╨──┐                   »
«     b: 1/═══════════════════════╡ 0x1 ╞═══════════════════»
«                                 └─────┘                   »
«Result: 1/═════════════════════════════════════════════════»
«                                                           »
«                                           
«       Q: ─────────────────────────────────
«                                           
«       A: ─────────────────────────────────
«          ┌────────────────────────────┐┌─┐
«       B: ┤ U(-4.1677,-2.0638,-4.2452) ├┤M├
«          └────────────────────────────┘└╥┘
«     a: 1/═══════════════════════════════╬═
«                                         ║ 
«     b: 1/═══════════════════════════════╬═
«                                         ║ 
«Result: 1/═══════════════════════════════╩═
«                                         0